In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
#fablib.show_config();

In [3]:
site1='CERN'
slice_name = 'slice_CERN'
node1_name = 'node1'

network1_name='net1'
network2_name='net2'

node1_nic1_name = 'nic1'
node1_nic2_name = 'nic2'


In [4]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_Basic', name=node1_nic1_name).get_interfaces()[0]
iface2 = node1.add_component(model='NIC_Basic', name=node1_nic2_name).get_interfaces()[0]

# NetworkS
net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv6')
net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv6')

#Submit Slice Request
slice.submit();


Retry: 13, Time: 323 sec


ID,327b18b0-fa6a-444d-9900-6f5aa8e38fdb
Name,slice_CERN
Lease Expiration (UTC),2024-07-23 22:23:59 +0000
Lease Start (UTC),2024-07-22 22:23:59 +0000
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
cd49d324-365e-4cfd-b2fd-927338e3ac4d,node1,2,8,10,default_rocky_8,qcow2,cern-w1.fabric-testbed.net,CERN,rocky,2001:400:a100:3090:f816:3eff:fe37:ed82,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe37:ed82,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
04ebd461-038f-464e-a307-f8f92530d42a,net1,L3,FABNetv6,CERN,2602:fcfb:1d:2::/64,2602:fcfb:1d:2::1,Active,
0f3e2d87-818f-425a-8180-9298a05eab7d,net2,L3,FABNetv6,CERN,2602:fcfb:1d:1::/64,2602:fcfb:1d:1::1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
node1-nic2-p1,p1,node1,net2,100,config,,0A:2B:1C:A6:7C:94,eth2,eth2,2602:fcfb:1d:1:2411:52c9:7279:dbfc,6,HundredGigE0/0/0/5
node1-nic1-p1,p1,node1,net1,100,config,,06:CD:3C:FF:8A:7B,eth1,eth1,2602:fcfb:1d:2:64c:d8c1:f889:1022,6,HundredGigE0/0/0/5



Time to print interfaces 333 seconds


In [ ]:
network2 = slice.get_network(name=network2_name)
network2.show()

In [28]:
network1_available_ips = network1.get_available_ips()
network2_available_ips = network2.get_available_ips()

In [29]:
node1_iface2 = node1.get_interface(network_name=network2_name)  
node1_addr2 = network2_available_ips.pop(0)

In [30]:
node1_iface2.ip_addr_add(addr=node1_addr2, subnet=network2.get_subnet())

Exception: node.execute: Management IP Invalid: None

In [31]:
node1_addr2

IPv6Address('2602:fcfb:1d:1::2')

In [32]:
network2.get_subnet()

IPv6Network('2602:fcfb:1d:1::/64')

In [7]:
nodes = slice.get_nodes()
print({nodes[0].get_ssh_command()})


{'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe37:ed82'}
